Привет! Меня зовут Исмаил, я буду проверять твой проект :) Можешь обращаться ко мне на «ты». Если тебе комфортно, то и я буду к тебе так обращаться, если нет, то обязательно скажи об этом.

Пожалуйста, не удаляй мои комментарии, которые я буду оставлять в работе. Это особенно поможет, если твои проекты будут отправлены на повторную проверку. При повторных проверках у комментариев будут приписки: "Вторая итерация", "Третья итерация" и т.д. 

Ты тоже можешь реагировать на мои комментарии, но в таком случае постарайся, чтобы твои комментарии отличались от моих: например, выделяй их своим любимым цветом — так у нас не возникнет путаницы :)

Кстати, про цвета! Мои комментарии будут в следующей цветовой гамме: зелеными, желтыми и красными. Например:

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 </b>
    
В случае, если решение на отдельном шаге является полностью правильным. </div>

<div class="alert-warning"> 
<b>Комментарий ревьюера 💡</b> 
    
В случае, если решение может стать еще лучше с некоторыми корректировками </div>
 
<div class="alert-danger"> 
<b>Комментарий ревьюера ❌</b> 
    
В случае, если решение какого-то шага требует значительных изменений. Проект не может быть принят с первого раза, если ревью содержит комментарии, помеченные этим цветом </div>

# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [2]:
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV


In [3]:
df = pd.read_csv('/datasets/geo_data_0.csv')


<div class="alert-success"> 
<b>Комментарий ревьюера 👍</b>

Хороший первичный анализ данных

## Разбейте данные на выборки

Разобьем выборку на три части обучающую, валидационную и тестовую, в отношении 3:1:1

In [2]:
df_train_base, df_test = train_test_split(df, test_size=0.2, random_state=12345)
df_train, df_valid = train_test_split(df_train_base, test_size=0.25, random_state=12345)
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

**Размер тестовой выборки** 

In [3]:
print(features_test.shape)
print(target_test.shape)

(643, 4)
(643,)


**Размер обучающей выборки**

In [4]:
print(features_train.shape)
print(target_train.shape)

(1928, 4)
(1928,)


**Размер валидационной выборки**

In [5]:
print(features_valid.shape)
print(target_valid.shape)

(643, 4)
(643,)


<div class="alert-success"> 
<b>Комментарий ревьюера 👍</b>

Выборки разделены верно, здорово, что проверены их размеры

## Исследуйте модели

**Дерево решений**

In [6]:
best_model = None
best_result = 0
for list_splitter in ['best','random']:
    for list_criterion in ['gini', 'entropy']:
        for depth in range(1, 20):
            model = DecisionTreeClassifier(random_state=12345, max_depth=depth, criterion = list_criterion, splitter=list_splitter)
            model.fit(features_train,target_train) 
            predictions = model.predict(features_valid)
            result = accuracy_score(target_valid,predictions) 
            if result > best_result:
                best_model = model
                best_depth = depth
                best_result = result
                best_criterion = list_criterion
                best_splitter = list_splitter
print("Accuracy лучшей модели дерева решений:", best_result,"Глубина дерева:",  best_depth, "Критерий качества разбиения:", best_criterion, "Стратегия разбиения:", best_splitter)


Accuracy лучшей модели дерева решений: 0.7838258164852255 Глубина дерева: 9 Критерий качества разбиения: entropy Стратегия разбиения: best


<div class="alert-danger"> 
<b>Комментарий ревьюера ❌</b> 

Обучение верное, но рассмотрен только один гиперпараметр. Хотелось бы видеть перебор значений и у других гиперпараметров

**Логистическая регрессия**

In [33]:
best_model = None
best_result = 0
#for list_solver in ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']:
for list_multi_class in ['auto', 'ovr', 'multinomial']:
     for list_max_iter in range(100, 500, 10):
        model = LogisticRegression(random_state=12345, multi_class = list_multi_class, max_iter=list_max_iter)
        model.fit(features_train,target_train) 
        predictions = model.predict(features_valid)
        result = accuracy_score(target_valid,predictions) 
        if result > best_result:
            best_model = model
            best_result = result
            best_multi_class = list_multi_class
            best_max_iter = list_max_iter
            
print("Accuracy лучшей модели дерева решений:", best_result,"multi_class:", best_multi_class, "max_iter:", best_max_iter)

Accuracy лучшей модели дерева решений: 0.7262830482115086 multi_class: auto max_iter: 100


**Подбор гиперпараметров для модели логистической регрессии с помощью GridSearchCV**

In [34]:
model_lr = LogisticRegression()
parametrs = {'multi_class': ['auto', 'ovr', 'multinomial'],
                    'max_iter': range (100, 500, 10), 
                    'random_state': [None, 12345]}
clf = GridSearchCV(model_lr, parametrs, n_jobs=-1, cv=5, refit=True, scoring='accuracy')
clf.fit(features_train, target_train) 
best_params = clf.best_params_

print('Лучшие гиперпараметры для логистической регрессии:', best_params)


Лучшие гиперпараметры для логистической регрессии: {'max_iter': 100, 'multi_class': 'multinomial', 'random_state': None}


In [35]:
model = LogisticRegression(random_state=None, max_iter=100, multi_class='multinomial' )
model.fit(features_train, target_train) 
predictions = model.predict(features_valid)
accuracy_lr = accuracy_score(predictions, target_valid)
print('Accuracy логистической регрессии:', accuracy_lr)

Accuracy логистической регрессии: 0.7262830482115086


<div class="alert-warning"> 
<b>Комментарий ревьюера 💡 </b>

Тут тоже хотелось бы видеть перебор значений параметров 

(и регрессия логистическая)

**Модель случайный лес**

In [17]:
best_model = None
best_result = 0
for est in range(1, 11):
    for depth in range(1, 10):
        for samples_leaf in range(1,8):
            for samples_split in range (2,10,2):
                model = RandomForestClassifier(
                random_state=12345, n_estimators=est, max_depth = depth, min_samples_leaf=samples_leaf, min_samples_split=samples_split) 
                model.fit(features_train,target_train) 
                result = model.score(features_valid,target_valid) 
                if result > best_result:
                    best_model = model
                    best_result = result
                    best_depth = depth
                    best_est = est
                    best_samples_leaf = samples_leaf
                    best_samples_split = samples_split
print("Accuracy лучшей модели случайного леса:", best_result, "Количество деревьев", best_est, "Глубина:",best_depth, "минимальное число образцов в листах", best_samples_leaf, "минимальное число образцов для сплита",best_samples_split)

Accuracy лучшей модели случайного леса: 0.7993779160186625 Количество деревьев 3 Глубина: 9 минимальное число образцов в листах 1 минимальное число образцов для сплита 8


In [16]:
model = RandomForestClassifier(random_state=12345, n_estimators=3, max_depth=9, min_samples_leaf=1, min_samples_split=8) 
model.fit(features_train,target_train) 
result = model.score(features_valid,target_valid) 
print("Accuracy лучшей модели случайного леса:", result)

Accuracy лучшей модели случайного леса: 0.7993779160186625


**Подбор гиперпараметров для модели случайного леса с помощью GridSearchCV**

In [18]:
model_RFC = RandomForestClassifier()
parametrs_RFC = { 'n_estimators': range (1, 11),
              'max_depth': range (1,10),
              'min_samples_leaf': range (1,8),
              'min_samples_split': range (2,10,2) }
clf_RFC = GridSearchCV(model_RFC, parametrs_RFC, cv=5)
clf_RFC.fit(features_train, target_train) 
best_params_RFC = clf_RFC.best_params_

print('Лучшие гиперпараметры для модели случайного леса:', best_params_RFC)

Лучшие гиперпараметры для модели случайного леса: {'max_depth': 9, 'min_samples_leaf': 5, 'min_samples_split': 8, 'n_estimators': 7}


In [20]:
model = RandomForestClassifier(random_state=12345, n_estimators=7, max_depth=9, min_samples_leaf=5, min_samples_split=8) 
model.fit(features_train,target_train) 
result = model.score(features_valid,target_valid) 
print("Accuracy модели случайного леса:", result)

Accuracy модели случайного леса: 0.7822706065318819


<div class="alert-danger"> 
<b>Комментарий ревьюера ❌</b> 

Аналогично с позапрошлым комментарием, у леса можно много чего глянуть

**Промежуточный вывод**

Лучше всего показала себя модель случайного леса с количеством деревьев равным 3, глубиной 9, минимальным числом образцов в листах 1, минимальным числом образцов для сплита 8 и Accuracy равным 0.7993779160186625

Подбор гиперпараметров с помощью GridSearchCV оказался менее точным с Accuracy равным 0.7822706065318819

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 Вторая итерация </b>

Теперь здорово, рассмотрено достаточное кол-во гиперпараметров. Чтобы циклы/подциклы перебора гиперпараметров не были такими громоздкими, можно испспользовать GridSearch (что ты и делаешь)

## Проверьте модель на тестовой выборке

In [36]:
model_rfc = RandomForestClassifier(random_state=12345, n_estimators=3, max_depth=9, min_samples_leaf=1, min_samples_split=8)
model_rfc.fit(features_train, target_train)
predictions_test_rfc = model_rfc.predict(features_test)
result_test_rfc = accuracy_score(predictions_test_rfc, target_test)
print('Accuracy RandomForestClassifier:', result_test_rfc)

Accuracy RandomForestClassifier: 0.7993779160186625


Accuracy у модели случайного леса на тестовой выборке равна 0.7993779160186625

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 </b>

Модель проверена на тесте

## (бонус) Проверьте модели на адекватность

In [22]:
model_dc = DummyClassifier(strategy='most_frequent', random_state=12345)
model_dc.fit(features_train, target_train)
result_dc = model_dc.score(features_valid, target_valid)
print('Accuracy DummyClassifier:', result_dc)

Accuracy DummyClassifier: 0.6889580093312597


Accuracy модели DummyClassifier ниже, чем у модели RandomForestClassifier - модель адекватна.

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 </b>

Можно проще, посмотреть на баланс классов: например, если у нас 90 процентов таргета это "1", а остальные "10" - 0, то мы можем тупо выдавать каждый раз цифру 1 и получать метрику 90 %. Любая модель, которая будет выдавать качество ниже этого - плохая/неадекватная

dummy classifier с strategy='most_frequent' работает так же: смотрит на самый часто встречающийся класс и выдает только его (только одно значение - константу, поэтому это константная модель)

## Вывод

В данном проекте мы рассмотрели три модели DecisionTreeClassifier, LogisticRegression, RandomForestClassifier.   
-Датафрейм был разделен на обучающую, тестовую и валидационную выборки в отношении 3:1:1.  
-Наилучшим образом показала себя модель RandomForestClassifier с гиперпараметрами random_state=12345, n_estimators=3, max_depth=9, min_samples_leaf=1, min_samples_split=8. Accuracy на тестовой выборке составила 0.793 
-Адекватность модели проверена с помощью модели DummyClassifier на тестовой выборке. Accuracy составила: 0.689


<div class="alert-danger"> 
<b>Комментарий ревьюера ❌</b>

Хороший проект, тобою проделана большая работа, ты молодец!

Сперва похвалю за наличие везде промежуточных выводов, комментариев. Хороший код, показываешь навыки владения sklearn. Все требуемые пункты в задании в целом выполнены. Оставил пару желтых комментариев, думаю, это сможет улучшить твой проект. Также можно добавить визуализацию в проект: посмотреть на графиках, как влияет перебор параметров на результат нашей метрики

В этом проекте большое внимание уделяется подбору гиперпараметров, но это можно автоматизировать! Советую почитать на досуге про GridsearchCV и Optuna, будет полезно
    
Также отправляю ссылки на документации некоторых моделей для классификации: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html, https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html, https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
    
Что обязательно стоит доработать: посмотреть побольше гиперпараметров у решающего дерева и случайного леса
    
Из курсов и дополнительной информации советую Yandex ML Handbook (https://academy.yandex.ru/handbook/ml) и курсы Евгения Соколова на ФКН (Машинное обучение - 1 с математикой https://github.com/esokolov/ml-course-hse и Основы машинного обучения без глубокой математики https://github.com/hse-ds/iad-intro-ds) - там можешь получить дополнительную информацию, более глубокую, возможно. http://wiki.cs.hse.ru/Заглавная_страница - тут можешь глянуть записи семинаров и лекций и другие материалы соответствующих курсов. 
    
Буду ждать доработок, удачи!

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 Вторая итерация </b>

Все критические ошибки исправлены, проект стал еще лучше, принимаю. Успехов в дальнейшей учебе!

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
